## LCEL 인터페이스


사용자 정의 체인을 가능한 쉽게 만들 수 있도록, 우리는 ["Runnable"](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable) 프로토콜을 구현했습니다. `Runnable` 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.
이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다.
표준 인터페이스에는 다음이 포함됩니다:

- [`stream`](#stream): 응답의 청크를 스트리밍합니다
- [`invoke`](#invoke): 입력에 대해 체인을 호출합니다
- [`batch`](#batch): 입력 목록에 대해 체인을 호출합니다

이들에는 해당하는 비동기 메소드도 있습니다:

- [`astream`](#async-stream): 비동기적으로 응답의 청크를 스트리밍합니다
- [`ainvoke`](#async-invoke): 비동기적으로 입력에 대해 체인을 호출합니다
- [`abatch`](#async-batch): 비동기적으로 입력 목록에 대해 체인을 호출합니다
- [`astream_log`](#async-stream-intermediate-steps): 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다

**입력 타입**과 **출력 타입**은 컴포넌트에 따라 다릅니다:

| 컴포넌트     | 입력 타입                                      | 출력 타입        |
| ------------ | ---------------------------------------------- | ---------------- |
| Prompt       | 사전                                           | PromptValue      |
| ChatModel    | 단일 문자열, 채팅 메시지 목록 또는 PromptValue | ChatMessage      |
| LLM          | 단일 문자열, 채팅 메시지 목록 또는 PromptValue | 문자열           |
| OutputParser | LLM 또는 ChatModel의 출력                      | 파서에 따라 다름 |
| Retriever    | 단일 문자열                                    | 문서 목록        |
| Tool         | 도구에 따라 단일 문자열 또는 사전              | 도구에 따라 다름 |

모든 runnable은 입력과 출력 **스키마**를 노출하여 입력과 출력을 검사합니다:

- [`input_schema`](#input-schema): Runnable의 구조에서 자동 생성된 입력 Pydantic 모델
- [`output_schema`](#output-schema): Runnable의 구조에서 자동 생성된 출력 Pydantic 모델

이 메소드들을 살펴보겠습니다. 이를 위해, 우리는 매우 간단한 PromptTemplate + ChatModel 체인을 만들 것입니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일

from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

이 코드는 `langchain` 라이브러리를 사용하여 챗봇 모델을 구성합니다. `ChatOpenAI` 클래스를 사용하여 OpenAI의 챗봇 모델 인스턴스를 생성하고, `ChatPromptTemplate` 클래스를 사용하여 특정 주제에 대한 농담을 요청하는 프롬프트 템플릿을 만듭니다. 이후 프롬프트와 모델을 파이프 연산자(`|`)를 사용하여 연결함으로써 대화 체인을 구성합니다. 사용자는 이 체인을 통해 주제를 입력하면 관련 농담을 생성하는 챗봇과 상호작용할 수 있습니다.


In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# ChatOpenAI 모델을 인스턴스화합니다.
model = ChatOpenAI()
# 주어진 토픽에 대한 농담을 요청하는 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
# 프롬프트와 모델을 연결하여 대화 체인을 생성합니다.
chain = prompt | model

## 입력 스키마

Runnable에 의해 수락된 입력들에 대한 설명입니다.
이것은 어떤 Runnable의 구조로부터 동적으로 생성된 Pydantic 모델입니다.
JSONSchema 표현을 얻기 위해 `.schema()`를 호출할 수 있습니다.


`chain` 객체의 `input_schema` 속성을 사용하여 체인의 첫 번째 부분인 프롬프트의 입력 스키마를 조회합니다. `schema()` 메서드를 호출하여 해당 스키마의 구조를 얻을 수 있습니다.


In [3]:
# 체인의 입력 스키마는 첫 번째 부분인 프롬프트의 입력 스키마입니다.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

이 코드는 JSON 스키마를 정의하고, 해당 스키마를 출력하는 기능을 구현합니다. `input_schema` 변수는 'name'과 'age' 필드를 가진 객체를 기술하는 스키마를 담고 있으며, 이 두 필드는 필수 요소입니다. 'name' 필드는 문자열 타입이고, 'age' 필드는 정수 타입으로, 최소값은 0입니다. `print_schema` 함수는 인자로 받은 스키마를 JSON 형식으로 콘솔에 출력합니다. 마지막으로 `print_schema` 함수는 `input_schema`를 인자로 받아 호출되어 스키마를 출력합니다.


In [8]:
import json


# 스키마를 출력하는 함수를 정의합니다.
def print_schema(schema):
    # 스키마를 JSON 형식으로 출력합니다.
    print(json.dumps(schema, indent=4))


# 입력 스키마를 정의합니다.
input_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "age": {"type": "integer", "minimum": 0},
    },
    "required": ["name", "age"],
}

# 스키마를 출력하는 함수를 호출합니다.
print_schema(input_schema)

{
    "type": "object",
    "properties": {
        "name": {
            "type": "string"
        },
        "age": {
            "type": "integer",
            "minimum": 0
        }
    },
    "required": [
        "name",
        "age"
    ]
}


이 함수는 `model` 객체의 입력 스키마를 반환합니다. `input_schema` 속성은 모델이 예상하는 입력 데이터의 구조를 정의하며, `schema()` 메서드를 호출하여 이를 JSON 형식으로 출력합니다.


In [9]:
# 모델의 입력 스키마를 출력합니다.
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'str

## 출력 스키마

Runnable에서 생성된 출력에 대한 설명입니다.
이것은 Runnable의 구조에서 동적으로 생성된 Pydantic 모델입니다.
`.schema()`를 호출하여 JSONSchema 표현을 얻을 수 있습니다.


이 코드는 체인(chain)의 출력 스키마를 확인하는 예제입니다. 여기서 체인은 여러 처리 단계를 연결한 것을 의미하며, 마지막 단계의 출력 스키마가 전체 체인의 출력 스키마가 됩니다. 예시에서는 `ChatModel`이 마지막 단계이며, 이 모델은 `ChatMessage`를 출력합니다. `chain.output_schema.schema()` 메서드를 호출하여 현재 체인의 출력 스키마를 조회할 수 있습니다.


In [10]:
# 체인의 출력 스키마는 마지막 부분의 출력 스키마입니다. 이 경우 ChatModel의 출력 스키마로, ChatMessage를 출력합니다.
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'AIMessage': {'title': 'AIMessage',
   'description': 'A Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'string'},
      {'type': 'array',
       'items': {'anyOf': [{'type': 'string'}, {'type': 'object'}]}}]},
    'additional_kwargs': {'title': 'Additional Kwargs', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'ai',
     'enum': ['ai'],
     'type': 'string'},
    'example': {'title': 'Example', 'default': False, 'type': 'boolean'}},
   'required': ['content']},
  'HumanMessage': {'title': 'HumanMessage',
   'description': 'A Message from a human.',
   'type': 'object',
   'properties': {'conten

## stream: 실시간 출력


이 함수는 `chain.stream` 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(`content`)을 즉시 출력합니다. `end=""` 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True` 인자는 출력 버퍼를 즉시 비우도록 합니다. 이는 스트리밍 데이터를 실시간으로 처리할 때 유용하게 사용됩니다.


In [11]:
# chain.stream 메서드를 사용하여 '멀티모달' 토픽에 대한 스트림을 생성하고 반복합니다.
for s in chain.stream({"topic": "멀티모달"}):
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(s.content, end="", flush=True)

멀티모달은 둘 이상의 다양한 형태의 정보를 동시에 사용하거나 통합하는 시스템이다. 이는 시각, 청각, 촉각 등 다양한 감각을 활용하여 사용자에게 정보를 제공한다. 멀티모달은 사용자의 편의성과 접근성을 높이는데 기여하며, 예를 들어 음성 안내와 함께 시각적인 지도를 제공하여 길을 안내하는 자동차 내비게이션 시스템이 이에 해당한다.

## invoke: 호출


`chain` 객체의 `invoke` 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다. 이 예제에서는 `topic` 키에 `'ChatGPT'` 값을 가진 딕셔너리를 `invoke` 메서드에 전달하고 있습니다.


In [12]:
# chain 객체의 invoke 메서드를 호출하고, 'ChatGPT'라는 주제로 딕셔너리를 전달합니다.
chain.invoke({"topic": "ChatGPT"})

AIMessage(content='ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델로, 사용자와 자연스러운 대화를 수행할 수 있다. GPT는 대규모의 텍스트 데이터를 학습하여 문맥을 파악하고 응답을 생성하는데 사용되며, 사용자의 질문이나 명령에 유연하게 대응할 수 있다. ChatGPT는 다양한 주제에 대해 대화를 나누거나 문제를 해결하는 등 다양한 활용 가능성을 가지고 있다.')

## batch: 배치(단위 실행)


함수 `chain.batch`는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리를 수행합니다. 이 예시에서는 두 개의 토픽, `ChatGPT`와 `Instagram`에 대한 처리를 요청합니다.


In [13]:
# 주어진 토픽 리스트를 batch 처리하는 함수 호출
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

[AIMessage(content='ChatGPT는 GPT-3 모델을 기반으로한 대화형 인공지능이다. 이 모델은 사용자와 자연스럽게 상호작용하며 다양한 주제에 대해 대화를 나눌 수 있다. ChatGPT는 질문에 답변하는 능력뿐만 아니라 지시에 따라 특정 작업을 수행하거나 문장을 생성하는 등 다양한 작업에 유용하게 사용될 수 있다.'),
 AIMessage(content='1. Instagram은 사진 및 동영상을 공유할 수 있는 소셜미디어 플랫폼으로, 사용자들은 자신의 일상이나 관심사를 시각적으로 표현하고 다른 사람들과 소통할 수 있다.\n\n2. 이 앱은 다양한 필터와 편집 기능을 제공하여 사용자들이 사진을 더욱 아름답게 보여줄 수 있으며, 해시태그를 통해 특정 주제나 관심사에 대한 게시물을 찾을 수도 있다.\n\n3. 인기 있는 인플루언서들이 많이 활동하는 Instagram은 광고 및 마케팅의 중요한 도구로 사용되며, 사용자들은 팔로워들과의 상호작용을 통해 자신의 영향력과 인지도를 키울 수 있다.')]

`max_concurrency` 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다


함수 `chain.batch`는 여러 작업을 동시에 처리할 수 있도록 배치 작업을 생성합니다. 이 함수는 두 개의 인자를 받습니다: 작업 목록과 설정을 담은 `config` 딕셔너리입니다. 작업 목록은 딕셔너리의 리스트로, 각 딕셔너리는 처리할 주제를 나타내는 `topic` 키를 포함합니다. `config` 딕셔너리는 `max_concurrency` 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.


In [14]:
chain.batch(
    [
        {"topic": "ChatGPT"},
        {"topic": "Instagram"},
        {"topic": "멀티모달"},
        {"topic": "프로그래밍"},
        {"topic": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

[AIMessage(content='ChatGPT는 OpenAI에서 개발한 자연어 처리 모델로, 다양한 대화 상황에서 사용자와 대화하는 역할을 수행한다. 이 모델은 사전에 대규모로 학습된 데이터를 기반으로 생성되었으며, 사용자의 입력에 적절한 응답을 제공하여 대화를 지속적으로 이어나갈 수 있다. ChatGPT는 자연어 이해와 생성 능력이 뛰어나며, 실제 사용자와의 상호작용을 통해 지속적으로 발전하고 있다.'),
 AIMessage(content='Instagram은 사진과 동영상을 공유하고 소셜 네트워크 기능을 이용할 수 있는 모바일 앱이다. 사용자들은 자신의 프로필에 사진과 동영상을 업로드하고 다른 사용자들과 인터랙션을 할 수 있다. 또한, 인기 있는 해시태그를 활용하여 다양한 주제의 게시물을 찾을 수 있다.'),
 AIMessage(content='멀티모달은 여러 가지 모드(예: 음성, 이미지, 텍스트)를 사용하여 정보를 전달하는 것이다.\n멀티모달은 다양한 감각을 동시에 사용하여 정보를 받는 사람의 이해를 돕는다.\n멀티모달은 시각과 청각 등 다양한 감각을 활용하여 보다 효과적인 의사소통과 정보 전달을 가능하게 한다.'),
 AIMessage(content='프로그래밍은 컴퓨터에게 실행할 일련의 명령을 작성하는 과정이다.\n이를 통해 우리는 문제를 해결하거나 원하는 결과를 얻을 수 있다.\n프로그래밍은 문제 해결 능력과 창의력을 키우는 동시에 다양한 분야에서 활용되는 핵심 기술이다.'),
 AIMessage(content='머신러닝은 컴퓨터 시스템이 데이터를 학습하고 패턴을 식별하여 예측, 분류, 클러스터링 등의 작업을 수행하는 인공지능의 한 분야입니다.\n\n머신러닝은 통계학, 컴퓨터 과학, 인공지능 등의 다양한 학문을 기반으로 하며, 데이터를 기반으로 모델을 학습시켜 새로운 데이터에 대한 예측을 수행합니다.\n\n머신러닝은 비정형 데이터의 처리에서 매우 유용하며, 의료진단, 금융분석, 영상처리 등 다양한 분야에서 활용되고 있습니다.')]

## async stream: 비동기 스트림


함수 `chain.astream`은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

비동기 for 루프(`async for`)를 사용하여 스트림에서 메시지를 순차적으로 받아오고, `print` 함수를 통해 메시지의 내용(`s.content`)을 즉시 출력합니다. `end=""`는 출력 후 줄바꿈을 하지 않도록 설정하며, `flush=True`는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.


In [15]:
# 비동기 스트림을 사용하여 'YouTube' 토픽의 메시지를 처리합니다.
async for s in chain.astream({"topic": "YouTube"}):
    # 메시지 내용을 출력합니다. 줄바꿈 없이 바로 출력하고 버퍼를 비웁니다.
    print(s.content, end="", flush=True)

YouTube는 구글이 운영하는 동영상 공유 플랫폼으로, 사용자들은 다양한 주제와 형식의 동영상을 업로드하고 시청할 수 있습니다. 또한 이용자들은 댓글과 좋아요 등의 반응을 표시하며, 구독을 통해 원하는 채널의 업데이트를 받을 수 있습니다. 유명인들의 뮤직비디오, 유머 콘텐츠, 교육자료 등 다양한 콘텐츠를 제공하여 많은 사람들에게 인기를 얻고 있습니다.

## async invoke: 비동기 호출


`chain` 객체의 `ainvoke` 메서드는 비동기적으로 주어진 인자를 사용하여 작업을 수행합니다. 여기서는 `topic`이라는 키와 `NVDA`(엔비디아의 티커) 라는 값을 가진 딕셔너리를 인자로 전달하고 있습니다. 이 메서드는 특정 토픽에 대한 처리를 비동기적으로 요청하는 데 사용될 수 있습니다.


In [16]:
# 비동기 체인 객체의 'ainvoke' 메서드를 호출하여 'NVDA' 토픽을 처리합니다.
await chain.ainvoke({"topic": "NVDA"})

AIMessage(content='NVDA는 NVIDIA Corporation의 주식을 나타내는 기호로, 세계적으로 유명한 그래픽 처리 장치 및 인공지능 기술 개발 회사이다. NVDA 주식은 기술 혁신과 성장 가능성으로 인해 많은 투자자들에게 인기를 끌고 있으며, 주가 상승으로 인한 수익을 기대할 수 있다. 또한, NVDA는 컴퓨터 게임, 자율주행차, 데이터 센터 및 클라우드 컴퓨팅 분야에서 혁신적인 기술을 개발하여 전 세계적으로 큰 영향력을 가지고 있다.')

## async batch: 비동기 배치


함수 `abatch`는 비동기적으로 일련의 작업을 일괄 처리합니다. 이 예시에서는 `chain` 객체의 `abatch` 메서드를 사용하여 `bears`라는 토픽에 대한 작업을 비동기적으로 처리하고 있습니다. `await` 키워드는 해당 비동기 작업이 완료될 때까지 기다리는 데 사용됩니다.


In [17]:
# 주어진 토픽에 대해 비동기적으로 일괄 처리를 수행합니다.
await chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

[AIMessage(content='YouTube는 구글이 소유하고 있는 동영상 공유 플랫폼이다. 사용자들은 자신의 동영상을 업로드하고 시청할 수 있으며, 다양한 주제와 장르의 동영상을 찾아볼 수 있다. 또한, 유명인들이나 크리에이터들이 자신의 콘텐츠를 공유하고 수익을 창출하는데에도 사용된다.'),
 AIMessage(content='Instagram은 사진과 동영상을 공유하고 소셜 네트워킹 서비스를 제공하는 애플리케이션입니다.\n사용자들은 자신의 프로필을 생성하고 팔로우, 좋아요, 댓글 등의 인터랙션을 통해 다른 사용자와 소통할 수 있습니다.\n인기 있는 인플루언서들이 많이 활동하는 플랫폼으로, 다양한 관심사와 스타일을 가진 사람들이 모여있어 다양한 콘텐츠를 즐길 수 있습니다.'),
 AIMessage(content='페이스북은 2004년에 설립된 소셜 네트워크 서비스로, 사용자들이 가입하여 친구와 사진, 동영상, 링크 등 다양한 콘텐츠를 공유하고 소통할 수 있는 플랫폼입니다. 전 세계적으로 약 29억 명의 활성 사용자를 보유하며, 개인 및 기업의 마케팅 및 광고에도 널리 활용되고 있습니다. 또한, 페이스북은 인스타그램과 워크플레이스 등 다른 서비스도 운영하고 있습니다.')]

## async stream 중간 단계 디버깅

모든 runnable은 체인/시퀀스의 중간 단계를 스트리밍하기 위해 사용되는 `.astream_log()` 메서드를 가지고 있습니다.

이것은 사용자에게 진행 상황을 보여주거나, 중간 결과를 사용하거나, 체인을 디버깅하는 데 유용합니다.

모든 단계를 스트리밍할 수도 있고(기본값), 이름, 태그 또는 메타데이터에 따라 단계를 포함하거나 제외할 수 있습니다.

이 메서드는 RunState를 구축하는 데 필요한 순서대로 적용될 때 [JSONPatch](https://jsonpatch.com) 작업을 생성합니다.

`LogEntry` 클래스와 `RunState` 클래스는 각각 서브-런과 런의 상태를 나타내는 필드와 설명을 포함하고 있습니다.


### JSONPatch 청크 스트리밍

이것은 예를 들어 HTTP 서버에서 `JSONPatch`를 스트리밍하고, 클라이언트에서 작업을 적용하여 실행 상태를 재구성하는 데 유용합니다. 웹서버를 더 쉽게 구축할 수 있도록 도와주는 도구인 [LangServe](https://github.com/langchain-ai/langserve)를 참조하세요.


이 코드는 `langchain` 라이브러리를 사용하여 텍스트 기반 질의응답 시스템을 구축하는 과정을 보여줍니다. 먼저 `OpenAIEmbeddings`를 사용하여 임베딩을 생성하고, `FAISS` 벡터 스토어를 이용해 텍스트를 인덱싱합니다. 그 후 `ChatPromptTemplate`를 사용하여 질문에 대한 프롬프트를 생성합니다. 이 프롬프트는 `retriever`와 `RunnablePassthrough`를 통해 질문의 컨텍스트를 검색하고, 질문을 그대로 전달합니다. `retrieval_chain`은 이러한 컴포넌트들을 파이프라인으로 연결하고, `StrOutputParser`를 사용하여 모델의 출력을 문자열로 파싱합니다. 마지막으로, `astream_log`를 비동기적으로 반복하여 질문에 대한 답변을 로그로 출력합니다.


In [24]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["테디가 살고 있는 곳은 대한민국 입니다."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

async for chunk in retrieval_chain.astream_log(
    "테디가 살고 있는 곳은 어딘가요?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '9e6620ed-6b2d-42e5-92fa-6cbc0fe88543',
            'logs': {},
            'streamed_output': []}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '720b0985-2569-4e21-9d9d-3691db02fbd0',
            'metadata': {},
            'name': 'Docs',
            'start_time': '2024-02-03T07:12:31.145+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
            'type': 'retriever'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs/final_output',
  'value': {'documents': [Document(page_content='테디가 살고 있는 곳은 대한민국 입니다.')]}},
 {'op': 'add',
  'path': '/logs/Docs/end_time',
  'value': '2024-02-03T07

### 증분 `RunState` 스트리밍

`diff=False`를 전달하면 `RunState`의 증분 값을 간단히 얻을 수 있습니다.
더 자세한 출력을 원하면 반복되는 부분이 더 많아집니다.


이 함수는 `retrieval_chain` 객체의 `astream_log` 비동기 메서드를 사용하여 로그 데이터를 비동기적으로 검색합니다. 검색 쿼리는 "where did harrison work?"이며, `include_names` 매개변수를 통해 "Docs"를 포함한 로그만을 필터링합니다. `diff` 매개변수는 `False`로 설정되어 있어, 변경 사항에 대한 차이점을 표시하지 않습니다. 각 로그 청크(chunk)가 검색될 때마다, 청크의 내용을 출력하기 전에 구분선을 출력합니다.


In [25]:
async for chunk in retrieval_chain.astream_log(
    "테디가 살고 있는 곳은 어딘가요?", include_names=["Docs"], diff=False
):
    print("-" * 70)
    print(chunk)

----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '333a7d22-215a-40a7-bf31-06d405e06eaa',
 'logs': {},
 'streamed_output': []})
----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '333a7d22-215a-40a7-bf31-06d405e06eaa',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': '00367c1d-5e3d-4dc1-9c25-09444ed6fda2',
                   'metadata': {},
                   'name': 'Docs',
                   'start_time': '2024-02-03T07:12:44.826+00:00',
                   'streamed_output': [],
                   'streamed_output_str': [],
                   'tags': ['map:key:context', 'FAISS', 'OpenAIEmbeddings'],
                   'type': 'retriever'}},
 'streamed_output': []})
----------------------------------------------------------------------
RunLog({'final_output': None,
 'id': '333a7d22-215a-40a7-bf31-06d405e06eaa',
 'logs'

## Parallel: 병렬성

LangChain Expression Language가 병렬 요청을 지원하는 방법을 살펴봅시다.
예를 들어, `RunnableParallel`을 사용할 때(자주 사전 형태로 작성됨), 각 요소를 병렬로 실행합니다.


이 코드는 `langchain_core.runnables` 모듈의 `RunnableParallel` 클래스를 사용하여 두 가지 작업을 병렬로 실행하는 예시를 보여줍니다. `ChatPromptTemplate.from_template` 메서드를 사용하여 주어진 `topic`에 대한 농담과 짧은 시를 생성하는 두 개의 체인(`chain1`, `chain2`)을 만듭니다. 이 체인들은 각각 `model`과 파이프(`|`) 연산자를 통해 연결됩니다. 마지막으로, `RunnableParallel` 클래스를 사용하여 이 두 체인을 `joke`와 `poem`이라는 키로 결합하여 동시에 실행할 수 있는 `combined` 객체를 생성합니다.


In [27]:
from langchain_core.runnables import RunnableParallel

# {country} 의 수도를 물어보는 체인을 생성합니다.
chain1 = ChatPromptTemplate.from_template("{country} 의 수도는 어디야?") | model

# {country} 의 면적을 물어보는 체인을 생성합니다.
chain2 = ChatPromptTemplate.from_template("{country} 의 면적은 얼마야?") | model
# 위의 2개 체인을 동시에 생성하는 병렬 실행 체인을 생성합니다.
combined = RunnableParallel(capital=chain1, area=chain2)

`chain1.invoke()` 함수는 `chain1` 객체의 `invoke` 메서드를 호출합니다. 이때, `topic`이라는 키에 `bears`라는 값을 가진 딕셔너리를 인자로 전달합니다. 이 메서드는 주로 체인 코드의 함수를 호출할 때 사용되며, 여기서는 'bears'라는 주제에 대한 처리를 요청합니다.


In [28]:
chain1.invoke(
    {"country": "대한민국"}
)  # chain1 객체의 invoke 메서드를 호출하고, 'country' 키에 '대한민국' 값을 전달합니다.

AIMessage(content='대한민국의 수도는 서울입니다.')

`chain2` 객체의 `invoke` 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다. 이 예시에서는 'bears'라는 문자열을 주제로 `invoke` 메서드를 호출하고 있습니다. `%%time` 매직 커맨드는 해당 셀의 실행 시간을 측정합니다.


In [29]:
# chain2 객체의 invoke 메서드를 호출하고, '미국'이라는 인자를 전달합니다.
chain2.invoke({"country": "미국"})

AIMessage(content='미국의 면적은 약 9,826,675 제곱 킬로미터입니다.')

`combined` 객체의 `invoke` 메서드는 주어진 `country`에 대한 처리를 수행합니다.

이 예제에서는 `대한민국`라는 주제를 `invoke` 메서드에 전달하여 실행합니다.


In [30]:
# 주어진 'country'에 대해 'combined' 객체의 'invoke' 메서드를 호출합니다.
combined.invoke({"country": "대한민국"})

{'capital': AIMessage(content='대한민국의 수도는 서울입니다.'),
 'area': AIMessage(content='대한민국의 면적은 약 100,363.4 제곱킬로미터입니다.')}

### 배치에서의 병렬 처리

병렬 처리는 다른 실행 가능한 코드와 결합될 수 있습니다.
배치와 병렬 처리를 사용해 보도록 합시다.


`chain1.batch` 함수는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 "topic" 키에 해당하는 값을 처리합니다. 이 예시에서는 "대한민국"와 "미국"라는 두 개의 토픽을 배치 처리하고 있습니다.


In [32]:
# 배치 처리를 수행합니다.
chain1.batch([{"country": "대한민국"}, {"country": "미국"}])

[AIMessage(content='대한민국의 수도는 서울입니다.'),
 AIMessage(content='미국의 수도는 워싱턴 D.C. (Washington D.C.)입니다.')]

`chain2.batch` 함수는 여러 개의 딕셔너리를 리스트 형태로 받아, 각 딕셔너리에 있는 `topic` 키의 값을 사용하여 일괄 처리(batch)를 수행합니다. 이 예시에서는 `bears`와 `cats`라는 두 가지 토픽에 대한 처리를 요청합니다.


In [33]:
# 배치 처리를 수행합니다.
chain2.batch([{"country": "대한민국"}, {"country": "미국"}])

[AIMessage(content='대한민국의 면적은 약 100,363 km²입니다.'),
 AIMessage(content='미국의 면적은 약 9,826,675km²입니다.')]

`combined.batch` 함수는 주어진 데이터를 배치로 처리하는 데 사용됩니다. 이 예시에서는 두 개의 딕셔너리 객체를 포함하는 리스트를 인자로 받아 각각 `대한민국`와 `미국` 두 나라에 대한 데이터를 배치 처리합니다.


In [34]:
# 주어진 데이터를 배치로 처리합니다.
combined.batch([{"country": "대한민국"}, {"country": "미국"}])

[{'capital': AIMessage(content='대한민국의 수도는 서울입니다.'),
  'area': AIMessage(content='대한민국의 면적은 약 100,363 제곱킬로미터입니다.')},
 {'capital': AIMessage(content='미국의 수도는 워싱턴 D.C. (District of Columbia) 입니다.'),
  'area': AIMessage(content='미국의 면적은 약 9,826,675 제곱 킬로미터입니다.')}]